Name: Wenhao Wu

ID: 1304952

In [2]:
import numpy as np
import keras
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [4]:
from keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

print(x_train.shape)
print(y_train.shape)

(404, 13)
(404,)
